In [22]:
from models.openvla_oft.configuration_prismatic import OpenVLAConfig
from models.openvla_oft.modeling_prismatic import OpenVLAForActionPrediction
from models.openvla_oft.processing_prismatic import PrismaticImageProcessor, PrismaticProcessor
# from models.openvla_oft.openvla_utils import update_auto_map, check_model_logic_mismatch
from transformers import AutoConfig, AutoImageProcessor, AutoModelForVision2Seq, AutoProcessor
AutoConfig.register("openvla", OpenVLAConfig)
AutoImageProcessor.register(OpenVLAConfig, PrismaticImageProcessor)
AutoProcessor.register(OpenVLAConfig, PrismaticProcessor)
AutoModelForVision2Seq.register(OpenVLAConfig, OpenVLAForActionPrediction)
# local_path = "/file_system/common-models/SimpleVLA-RL/Openvla-oft-SFT-libero10-trajall"
# local_path = "/file_system/common-models/SimpleVLA-RL/Openvla-oft-SFT-libero10-trajall"
# local_path = "/file_system/common-models/Haozhan72-kangsheng/Openvla-oft-SFT-libero10-trajall/"
# local_path = "/file_system/common-models/Haozhan72-kangsheng/Openvla-oft-SFT-libero10-traj1/"
local_path = "/file_system/kangsheng/verl/checkpoints/simple-vla-all-sft/merge/"
# if self.rank == 0:
#update_auto_map(local_path)
#check_model_logic_mismatch(local_path)

In [23]:
from models.openvla_oft.configuration_prismatic import OpenVLAConfig
from models.openvla_oft.processing_prismatic import PrismaticImageProcessor, PrismaticProcessor
print("*********USE VLA tokenizer*************")
AutoConfig.register("openvla", OpenVLAConfig)
AutoProcessor.register(OpenVLAConfig, PrismaticProcessor)
processor = AutoProcessor.from_pretrained(local_path, trust_remote_code=True)
tokenizer=processor.tokenizer

*********USE VLA tokenizer*************


In [24]:
import os
import torch
import json
torch_dtype = torch.float32

# override model kwargs
actor_model_config = AutoConfig.from_pretrained(local_path, trust_remote_code=True)

actor_module = AutoModelForVision2Seq.from_pretrained(
                                        pretrained_model_name_or_path=local_path,
                                        torch_dtype=torch_dtype,
                                        #attn_implementation="flash_attention_2",
                                        config=actor_model_config,              
                                        trust_remote_code=True,
                                    )
print(actor_module)
#oft add
actor_module.vision_backbone.set_num_images_in_input(1)

dataset_statistics_path = os.path.join(local_path, "dataset_statistics.json")
if os.path.isfile(dataset_statistics_path):
    with open(dataset_statistics_path, "r") as f:
        norm_stats = json.load(f)
    actor_module.norm_stats = norm_stats
else:
    print(
        "WARNING: No local dataset_statistics.json file found for current checkpoint.\n"
        "You can ignore this if you are loading the base VLA (i.e. not fine-tuned) checkpoint."
        "Otherwise, you may run into errors when trying to call `predict_action()` due to an absent `unnorm_key`."
    )

09/29 [06:57:58] WARNING  | >> Expected `transformers==4.40.1` and `tokenizers==0.19.1`   ]8;id=896568;file:///root/.cache/huggingface/modules/transformers_modules/modeling_prismatic.py\modeling_prismatic.py]8;;\:]8;id=452700;file:///root/.cache/huggingface/modules/transformers_modules/modeling_prismatic.py#332\332]8;;\
                          but got `transformers==4.55.4` and `tokenizers==0.21.4`; there                           
                          might be inference-time regressions due to dependency changes.                           
                          If in doubt, pleaseuse the above versions.                                               

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.35it/s]

OpenVLAForActionPrediction(
  (vision_backbone): PrismaticVisionBackbone(
    (featurizer): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=1024, out_features=3072, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=1024, out_features=1024, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): LayerScale()
          (drop_path1): Identity()
          (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
          (mlp)

In [25]:
import numpy as np
from env.libero_env import LiberoEnv
from env.libero_utils import get_libero_image, quat2axisangle
def _obs_to_input(obs):
    # remove the wrist image
    return {
        "full_image": get_libero_image(obs, 224),
        "state": np.concatenate([
            obs["robot0_eef_pos"],
            quat2axisangle(obs["robot0_eef_quat"]),
            obs["robot0_gripper_qpos"]
        ])
    }

In [26]:
from PIL import Image
import torchvision.transforms.functional as F
from torch.nn.utils.rnn import pad_sequence
def center_crop_image(image: Image.Image) -> Image.Image:

    crop_scale = 0.9
    orig_w, orig_h = image.size
    image_tensor = F.to_tensor(image)
    crop_h = int(orig_h * crop_scale)
    crop_w = int(orig_w * crop_scale)
    image_tensor = F.center_crop(image_tensor, (crop_h, crop_w))
    image_tensor = F.resize(image_tensor, (orig_h, orig_w))
    final_image = F.to_pil_image(image_tensor)
    
    final_image = final_image.convert("RGB")
    return final_image

def process_input(inputs:list, task_descriptions:list, config):
    
    batchdata = {"input_ids":[],"attention_mask":[],"pixel_values":[]}  
    
    for i in range(len(inputs)):
        input = inputs[i]
        task_description = task_descriptions[i]
        
        image = Image.fromarray(input["full_image"]).convert("RGB")
        if config["center_crop"]:
            image = center_crop_image(image)
        prompt = f"In: What action should the robot take to {task_description.lower()}?\nOut:"
        batch_feature  = processor(prompt, image)
        
        if "wrist_image" in input.keys():
            wrist_image = Image.fromarray(input["wrist_image"]).convert("RGB")
            if config["center_crop"]:
                wrist_image = center_crop_image(wrist_image)
            wrist_batch_feature = processor(prompt, wrist_image)
            primary_pixel_values = batch_feature["pixel_values"]
            batch_feature["pixel_values"] = torch.cat([primary_pixel_values] + [wrist_batch_feature["pixel_values"]], dim=1)
            
        input_ids = batch_feature["input_ids"]
        attention_mask = batch_feature["attention_mask"]
        pixel_values = batch_feature["pixel_values"]
        
        if not torch.all(input_ids[:, -1] == 29871):
            input_ids = torch.cat(
                (input_ids, torch.unsqueeze(torch.Tensor([29871]).long(), dim=0).to(input_ids.device)), dim=1
            )
            attention_mask = torch.cat(
                (attention_mask, torch.unsqueeze(torch.Tensor([True]).bool(), dim=0).to(attention_mask.device)), dim=1
            )
        
        batchdata["input_ids"].append(input_ids)    
        batchdata["attention_mask"].append(attention_mask)    
        batchdata["pixel_values"].append(pixel_values)    
    
    
    device = torch.device('cuda') 
    
    batchdata["input_ids"] = [x.transpose(0, 1) for x in batchdata["input_ids"]]
    batchdata["attention_mask"] = [x.transpose(0, 1) for x in batchdata["attention_mask"]]
    batchdata["input_ids"] = pad_sequence(batchdata["input_ids"], batch_first=True, padding_value=processor.tokenizer.pad_token_id).squeeze(-1).to(device)
    batchdata["attention_mask"] = pad_sequence(batchdata["attention_mask"], batch_first=True, padding_value=0).squeeze(-1).to(device)
    
    padding_mask = batchdata["input_ids"].ne(processor.tokenizer.pad_token_id)
    assert  torch.all(padding_mask==batchdata["attention_mask"].ne(0))
    padding_mask = ~padding_mask
    padding_mask = padding_mask.int() 
    sorted_indices = torch.argsort(padding_mask, dim=1, descending=True, stable=True)
    batchdata["input_ids"] = torch.gather(batchdata["input_ids"], 1, sorted_indices)
    batchdata["attention_mask"] = torch.gather(batchdata["attention_mask"], 1, sorted_indices)
    
    
    batchdata["pixel_values"] = torch.cat(batchdata["pixel_values"] , dim=0).to(device)
    assert torch.all(batchdata["attention_mask"].ne(0) == batchdata["input_ids"].ne(processor.tokenizer.pad_token_id))

    return batchdata
    

In [ ]:
actor_module = actor_module.to('cuda')
actor_module.eval()
actor_module.device

In [ ]:
# DO_SAMPLE = True
DO_SAMPLE = False
TEMP = 1.6
# TEMP = 0.2
UNNORM_KEY = "libero_10"
UNNORM_KEY = f"{UNNORM_KEY}_no_noops"
MAX_PROMPT_LENGTH = 512
def pad_sequence_to_length(tensors, max_seq_len, pad_token_id, left_pad=False):
    """
    pad a 2D tensors (e.g. responses, logprobs) in the last dim to max_seq_length.
    input shape: [bs, seq_length]
    output shape: [bs, max_seq_length]
    (0, max_seq_len - tensors.shape[-1]) means right pad to max_seq_length and no left pad
    """
    if tensors.shape[-1] >= max_seq_len:
        return tensors
    pad_tuple = (max_seq_len - tensors.shape[-1], 0) if left_pad else (0, max_seq_len - tensors.shape[-1])
    return torch.nn.functional.pad(tensors, pad_tuple, 'constant', pad_token_id)

@torch.no_grad()
def _generate_one_step(prompts: dict):
    idx = prompts['input_ids']  # (bs, prompt_length)
    attention_mask = prompts['attention_mask']  # left-padded attention_mask
    pixel_values = prompts["pixel_values"]



    # make sampling args can be overriden by inputs
    do_sample = prompts.get('do_sample', DO_SAMPLE)


    temperature = prompts.get('temperature', TEMP)

    #generation_config = GenerationConfig(temperature=temperature, top_p=top_p, top_k=top_k)

    with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
        actions, response = actor_module.generate_action_verl(
            input_ids=idx,
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            padding_idx = processor.tokenizer.pad_token_id,
            do_sample=do_sample,
            unnorm_key= UNNORM_KEY,
            temperature=temperature, )
    
    
    assert processor.tokenizer.pad_token_id is not None

    assert idx.ndim == 2
    idx = pad_sequence_to_length(idx,max_seq_len=MAX_PROMPT_LENGTH,pad_token_id=processor.tokenizer.pad_token_id,left_pad=True)
    
    assert attention_mask.ndim == 2
    attention_mask = pad_sequence_to_length(attention_mask,max_seq_len=MAX_PROMPT_LENGTH,pad_token_id=0,left_pad=True)
    
    
    assert idx.device.type == 'cuda'
    assert response.device.type == 'cuda'
    #assert seq.device.type == 'cuda'
    assert attention_mask.device.type == 'cuda'
    assert pixel_values.device.type == 'cuda'
    batch ={
            'responses': response,
            'input_ids': idx,
            'attention_mask': attention_mask,
            "pixel_values":pixel_values,
            "action":actions,
        }

    return batch

In [ ]:
# from collections import defaultdict
# import copy
# import torch.cuda.profiler as profiler
# from tqdm import trange

# config = {
#     "center_crop": True,
#     "num_steps_wait": 10
# }

# batch_size = 32
# # max_steps = 200
# max_steps = 512
# # max_steps = 1024
# # max_steps = 256
# action_chunks_len = 8
# # with torch.profiler.profile(
# #             activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
# #             profile_memory=True,  # 内存数据采集的开关
# #             record_shapes=True,  # 算子input shape信息采集的开关
# #             with_stack=True,
# #             with_flops=True,
# #             with_modules=True,
# #             schedule=torch.profiler.schedule(wait=10, warmup=1, active=3, repeat=2),
# #             on_trace_ready=torch.profiler.tensorboard_trace_handler("./traces")
# #     ) as prof:
# if True:
#     # libero_env = LiberoEnv(task_name="libero_goal", task_id=1, trial_id=0, is_valid=True, max_steps=50, config=config)
#     libero_env = LiberoEnv(task_name="libero_10", task_id=2, trial_id=0, is_valid=True, max_steps=max_steps, config=config)
#     valid_video = defaultdict(list)
#     vla_history = []
#     init_data = libero_env.get_initial_state()
#     print("Initial data:", init_data)
#     task_descriptions = [init_data['task_description']]
#     print(f"Task description: {task_descriptions}")

#     valid_video[init_data['task_file_name']].extend(init_data['valid_images'])
#     env_data = copy.deepcopy(init_data)
#     env_obs = env_data['obs']
#     for step in trange(max_steps // action_chunks_len):
#         # print("Step:", step)
#         # prof.step()
#         inputs = [_obs_to_input(env_obs)]
#         vla_input = process_input(inputs, task_descriptions, config)
#         # vla_input.update(meta_info)
#         vla_output = _generate_one_step(vla_input)
#         actions = vla_output["action"]
#         step_data = {
#             "responses": vla_output["responses"],
#             "input_ids": vla_output["input_ids"],
#             "attention_mask": vla_output["attention_mask"],
#             "pixel_values": vla_output["pixel_values"],
#             "action": actions,
#             "step": step
#         }
#         vla_history.append(step_data)
        

#         result = libero_env.step(actions[0])
#         valid_video[init_data['task_file_name']].extend(result['valid_images'])
#         env_obs = result["obs"]

#         step += action_chunks_len
#         complete = result["complete"]
#         if complete:
#             print(f"Task completed at step {step}.")
#             break
    

[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[Warning]: datasets path /file_system/cyk/vla_mix/LIBERO/libero/libero/../datasets does not exist!
[Warning]: datasets path /file_system/cyk/vla_mix/LIBERO/libero/libero/../datasets does not exist!
Initial data: {'type': 'init', 'obs': OrderedDict([('robot0_joint_pos', array([ 0.01827163, -0.16448719,  0.02169336, -2.40270198,  0.00596934,
        2.18337463,  0.81767892])), ('robot0_joint_pos_cos', array([ 0.99983308,  0.98650246,  0.99976471, -0.73921611,  0.99998218,
       -0.57497885,  0.68391642])), ('robot0_joint_pos_sin', array([ 0.01827062, -0.16374646,  0.02169166, -0.67346829,  0.0059693 ,
        0.81816827,  0.72956037])), ('robot0_joint_vel', array([-0.01455697,  0.02580653,  0.01168757,  0.0449542 ,  0.00851403,
       -0.0541633 ,  0.01900455])), ('robot0_eef_pos', array([-0.20136824,  0.01869972,  1.18931828])), ('robot0_eef_quat', array([ 9.99622153e-01,  2.13634188e-03, -2.74017070e-02,  3.65594565e-04])), ('rob

100%|██████████| 64/64 [02:31<00:00,  2.36s/it]


In [8]:
from env.libero_env import SubprocVecEnv
from collections import defaultdict
import copy
from tqdm import trange
config = {
    "center_crop": True,
    "num_steps_wait": 10
}
batch_size = 32
max_steps = 512
action_chunks_len = 8
# ------------------------------
# 1. 创建环境构造函数列表
# ------------------------------
# 使用 lambda 函数延迟环境的创建，直到子进程中才真正执行
env_fns = [
    lambda i=i: LiberoEnv(
        task_name="libero_10",
        task_id=0,
        trial_id=i,  # 每个环境 trial_id 不同
        is_valid=True,
        max_steps=max_steps,
        config=config
    )
    for i in range(batch_size)
]
# ------------------------------
# 2. 初始化向量化环境
# ------------------------------
vec_env = SubprocVecEnv(env_fns)

# 获取所有环境的初始状态
init_datas = vec_env.get_initial_states()
print(f"Initial data (env 0): {init_datas[0]}")
# 初始化任务描述、观测、视频存储和活跃状态
task_descriptions = [data['task_description'] for data in init_datas]
env_obs = [data['obs'] for data in init_datas]

valid_video = defaultdict(list)
for env_idx, data in enumerate(init_datas):
    key = f"{data['task_file_name']}_env{env_idx}"
    valid_video[key].extend(data['valid_images'])
    
dones = [False] * batch_size
vla_history = [] 
# ------------------------------
# 3. 主循环
# ------------------------------
total_chunks = max_steps // action_chunks_len
for chunk_idx in trange(total_chunks):
    active_envs_indices = [i for i, done in enumerate(dones) if not done]
    
    if not active_envs_indices:
        print("All environments completed!")
        break
    # 3.1 批量生成动作 (仅为活跃环境)
    active_obs = [env_obs[i] for i in active_envs_indices]
    active_task_desc = [task_descriptions[i] for i in active_envs_indices]
    inputs = [_obs_to_input(obs) for obs in active_obs]
    vla_input = process_input(inputs, active_task_desc, config)
    vla_output = _generate_one_step(vla_input)
    actions = vla_output["action"]  # shape: (num_active, action_chunks_len, ...)
    # 3.2 在子进程中并行执行动作
    # actions 的数量必须和 active_envs_indices 的数量匹配
    results = vec_env.step(actions, active_envs_indices)
    # 3.3 更新状态、视频和完成标志
    for i, env_idx in enumerate(active_envs_indices):
        result = results[env_idx] # 从完整结果列表中获取对应环境的结果
        
        # 更新观测
        env_obs[env_idx] = result["obs"]
        
        # 更新视频
        key = f"{init_datas[env_idx]['task_file_name']}_env{env_idx}"
        valid_video[key].extend(result['valid_images'])
        
        # 更新完成状态
        if result["complete"]:
            dones[env_idx] = True
            total_step = (chunk_idx + 1) * action_chunks_len
            print(f"Env {env_idx} completed at step ~{total_step}!")
# ------------------------------
# 4. 清理
# ------------------------------
print("Closing all environments...")
vec_env.close()
print("Done!")


[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9][Warning]: datasets path /file_system/cyk/vla_mix/LIBERO/libero/libero/../datasets does not exist!

[Warning]: datasets path /file_system/cyk/vla_mix/LIBERO/libero/libero/../datasets does not exist![Warning]: datasets path /file_system/cyk/vla_mix/LIBERO/libero/libero/../datasets does not exist![info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9][Warning]: datasets path /file_system/cyk/vla_mix/LIBERO/libero/libero/../datasets does not exist![Warning]: datasets path /file_system/cyk/vla_mix/LIBERO/libero/libero/../datasets does not exist!


[info] using task orders [0, 1, 2, 3, 4, 5, 6, 7, 8, 9][Warning]: datasets path /file_system/cyk/vla_mix/LIBERO/libero/libero/../datasets does not exist![Warning]: datasets path /file_system/cyk/vla_mix/LIBERO/libero/libero/../datasets does not exist!


[Warning]: datasets path /f

  0%|          | 0/64 [00:00<?, ?it/s]/file_system/cyk/vla_mix/test_rollout/.venv/lib/python3.11/site-packages/torchvision/transforms/functional.py:324: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(npimg, mode=mode)
 50%|█████     | 32/64 [01:28<01:28,  2.77s/it]

Env 30 completed at step ~256!


 55%|█████▍    | 35/64 [01:37<01:22,  2.83s/it]

Env 31 completed at step ~280!


...

Env 16 completed at step ~392!


 83%|████████▎ | 53/64 [02:21<00:24,  2.24s/it]

Env 23 completed at step ~424!


100%|██████████| 64/64 [02:45<00:00,  2.59s/it]


Closing all environments...
Done!


In [13]:
# len(valid_video)
valid_video.keys()

dict_keys(['libero_10_task_0_trial_0_env0', 'libero_10_task_0_trial_1_env1', 'libero_10_task_0_trial_2_env2', 'libero_10_task_0_trial_3_env3', 'libero_10_task_0_trial_4_env4', 'libero_10_task_0_trial_5_env5', 'libero_10_task_0_trial_6_env6', 'libero_10_task_0_trial_7_env7', 'libero_10_task_0_trial_8_env8', 'libero_10_task_0_trial_9_env9', 'libero_10_task_0_trial_10_env10', 'libero_10_task_0_trial_11_env11', 'libero_10_task_0_trial_12_env12', 'libero_10_task_0_trial_13_env13', 'libero_10_task_0_trial_14_env14', 'libero_10_task_0_trial_15_env15', 'libero_10_task_0_trial_16_env16', 'libero_10_task_0_trial_17_env17', 'libero_10_task_0_trial_18_env18', 'libero_10_task_0_trial_19_env19', 'libero_10_task_0_trial_20_env20', 'libero_10_task_0_trial_21_env21', 'libero_10_task_0_trial_22_env22', 'libero_10_task_0_trial_23_env23', 'libero_10_task_0_trial_24_env24', 'libero_10_task_0_trial_25_env25', 'libero_10_task_0_trial_26_env26', 'libero_10_task_0_trial_27_env27', 'libero_10_task_0_trial_28_en

In [19]:
# len(valid_video['libero_10_task_0_trial_0_env0'])
succeessful_videos = {key: valid_video[key] for key in valid_video if len(valid_video[key]) < 513}

In [20]:
len(succeessful_videos)

14

In [48]:
# vla_history

In [49]:
print(valid_video.keys())

dict_keys(['libero_goal_task_0_trial_0'])


In [14]:
import imageio
import random
def save_rollout_video(rollout_images, exp_name, task_name, step_idx, success ):
    """Saves an MP4 replay of an episode."""
    rollout_dir = f"./rollouts/{exp_name}" 
    os.makedirs(rollout_dir, exist_ok=True)
    ran_id = random.randint(1, 10000)
    #processed_task_description = task_description.lower().replace(" ", "_").replace("\n", "_").replace(".", "_")[:50]
    mp4_path = f"{rollout_dir}/step={step_idx}--task={task_name}--success={success}--ran={ran_id}.mp4"
    video_writer = imageio.get_writer(mp4_path, fps=30)
    for img in rollout_images:
        video_writer.append_data(img)
    video_writer.close()
    print(f"Saved rollout MP4 at path {mp4_path}")
    return mp4_path

In [21]:
for task_file, images in succeessful_videos.items():
    # complete = any(r['complete'] for r in task_records if r['task_file_name'] == task_file)
    complete = True
    save_rollout_video(
        images,
        "0",
        task_file,
        0,
        complete
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saved rollout MP4 at path ./rollouts/0/step=0--task=libero_10_task_0_trial_6_env6--success=True--ran=8333.mp4


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saved rollout MP4 at path ./rollouts/0/step=0--task=libero_10_task_0_trial_10_env10--success=True--ran=3105.mp4


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Saved rollout MP4 at path ./rollouts/0/step=0--task=libero_10_task_0_trial_28_env28--success=True--ran=9492.mp4


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saved rollout MP4 at path ./rollouts/0/step=0--task=libero_10_task_0_trial_30_env30--success=True--ran=6234.mp4


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Saved rollout MP4 at path ./rollouts/0/step=0--task=libero_10_task_0_trial_31_env31--success=True--ran=2258.mp4
Saved rollout MP4 at path ./rollouts/0/step=0--task=0--success=True--ran=526.mp4
Saved rollout MP4 at path ./rollouts/0/step=0--task=1--success=True--ran=4496.mp4
